In [1]:
import pandas as pd
import numpy as np
import names
import random
import matplotlib.pyplot as plt
import seaborn as sns
from randomtimestamp import randomtimestamp, random_date, random_time
from datetime import datetime
from datetime import timedelta
# Set figure size to (28,14)
plt.rcParams['figure.figsize'] = (28, 14)


In [2]:
#Display the number of customers at checkout (any state) over time

def question_03(df):
    df = pd.DataFrame(df.groupby(['location']).count()['customer_no'])
    return print(df)

In [3]:
#5 Calculate the total number of customers present in the supermarket over time.

def question_05(df):
    df = pd.DataFrame(df.groupby('timestamp').count()['customer_no'])
    sns.lineplot(data=df)

In [4]:
def question_04(df):
    df.reset_index(inplace=True)
    dict = {}
    # df_first = pd.DataFrame(columns=df.columns)
    # df_second = pd.DataFrame(columns=df.columns)
    for i in range(1,df['customer_no'].max(),1):
        #print(i)
        print(df[df['customer_no'] == i].iloc[0, 0])
        a = df[df['customer_no'] == i].iloc[-1, 0] - \
            df[df['customer_no'] == i].iloc[0, 0]
        dict[i] = a
        # df_first = df_first.append(a)
        # #print(df_first.head())
        # b = df[df['customer_no'] == i].iloc[1:]
        # df_second = df_second.append(b)
    # fig, axs = plt.subplots(2)
    # sns.histplot(df_first, x="location", ax=axs[0])
    # sns.histplot(df_second, x='location', ax=axs[1])
    return dict

In [5]:
# dict = question_06(df_0)

In [6]:
# Create customer_id
def create_id(df, letters):
    df['customer_id'] = letters + '_' + df['customer_no'].astype(str)

In [7]:
# Function for sorting
def sort_df(df):
    df.sort_index(inplace=True)
    df.sort_values('customer_no', kind='stable', inplace=True)
    return df


In [8]:
# Function for filling
def filling_steps(df):
    df_n = df.groupby('customer_no').resample('1T').ffill()
    df_l = df_n[['customer_no', 'location']].set_index('customer_no')
    df_l.reset_index(inplace=True)
    return df_l


In [9]:
# Function for adding checkouts to customers without
def checkout(df):
    for i in range(1, df['customer_no'].max()+1, 1):
        if df[df['customer_no'] == i].iloc[-1, 1] != 'checkout':
            a = df[df['customer_no'] == i].iloc[0,0]
            a_df = pd.DataFrame({"customer_no":[a],'location':'checkout'})
            df = df.append(a_df, ignore_index=True)
    return sort_df(df)

In [10]:
# create Markov columns
def markov_columns(df):
    df["before"] = df['location']
    df['after'] = df['location'].shift(-1)
    df.loc[df['before'] == 'checkout', 'after'] = 'checkout'
    df.set_index('customer_id', inplace=True)
    df.drop(columns=['customer_no','location'], inplace=True)
    return df


In [11]:
df_0 = pd.read_csv('data/monday.csv', parse_dates=True, index_col=0, sep=';')
df_1 = pd.read_csv('data/tuesday.csv', parse_dates=True, index_col=0, sep=';')
df_2 = pd.read_csv('data/wednesday.csv', parse_dates=True, index_col=0, sep=';')
df_3 = pd.read_csv('data/thursday.csv', parse_dates=True, index_col=0, sep=';')
df_4 = pd.read_csv('data/friday.csv', parse_dates=True, index_col=0, sep=';')


In [12]:
df_lst = [df_0, df_1, df_2, df_3, df_4]
days = ['MO','TU','WE','TH','FR']

In [13]:
for i in range(len(df_lst)):
    df_lst[i] = sort_df(df_lst[i])  # df_0 = sort_df(df_0)
    df_lst[i] = filling_steps(df_lst[i])
    df_lst[i] = checkout(df_lst[i])
    create_id(df_lst[i],days[i])   # create_id(df_0, 'MO')


In [14]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4884 entries, 2019-09-02 07:03:00 to 2019-09-02 21:50:00
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_no  4884 non-null   int64 
 1   location     4884 non-null   object
dtypes: int64(1), object(1)
memory usage: 114.5+ KB


In [15]:
df_0.head(20)

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:05:00,1,checkout
2019-09-02 07:03:00,2,dairy
2019-09-02 07:06:00,2,checkout
2019-09-02 07:04:00,3,dairy
2019-09-02 07:06:00,3,checkout
2019-09-02 07:04:00,4,dairy
2019-09-02 07:08:00,4,checkout
2019-09-02 07:04:00,5,spices


In [16]:
df_lst[1].tail(10)

,customer_no,location,customer_id
10114,1420,drinks,TU_1420
10115,1420,drinks,TU_1420
10116,1420,drinks,TU_1420
10117,1420,checkout,TU_1420
10118,1421,fruit,TU_1421
10119,1421,drinks,TU_1421
10120,1421,checkout,TU_1421
10121,1422,spices,TU_1422
10122,1422,dairy,TU_1422
10124,1422,checkout,TU_1422


In [17]:
#we create a list with every single df in order to concatenate them together
df = pd.concat(df_lst)
df.head(50)


,customer_no,location,customer_id
0,1,dairy,MO_1
1,1,dairy,MO_1
2,1,checkout,MO_1
3,2,dairy,MO_2
4,2,dairy,MO_2
5,2,dairy,MO_2
6,2,checkout,MO_2
7,3,dairy,MO_3
8,3,dairy,MO_3
9,3,checkout,MO_3


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53931 entries, 0 to 11075
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_no  53931 non-null  int64 
 1   location     53931 non-null  object
 2   customer_id  53931 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [19]:
df.tail(30)

,customer_no,location,customer_id
11044,1500,fruit,FR_1500
11045,1500,fruit,FR_1500
11046,1500,dairy,FR_1500
11070,1500,checkout,FR_1500
11047,1501,fruit,FR_1501
11048,1501,fruit,FR_1501
11049,1501,checkout,FR_1501
11050,1502,fruit,FR_1502
11051,1502,checkout,FR_1502
11052,1503,dairy,FR_1503


In [20]:
# General eda 
df['customer_id'].value_counts()

FR_102     53
TH_1047    53
WE_710     52
MO_666     52
FR_1238    50
           ..
WE_490      2
MO_928      2
TH_29       2
FR_1504     2
MO_131      2
Name: customer_id, Length: 7445, dtype: int64

In [21]:
df_mar = markov_columns(df)
df_mar.head()


,before,after
customer_id,,
MO_1,dairy,dairy
MO_1,dairy,checkout
MO_1,checkout,checkout
MO_2,dairy,dairy
MO_2,dairy,dairy


In [22]:
df_mar.tail(50)

,before,after
customer_id,,
FR_1497,dairy,dairy
FR_1497,dairy,dairy
FR_1497,dairy,checkout
FR_1497,checkout,checkout
FR_1498,fruit,fruit
FR_1498,fruit,checkout
FR_1498,checkout,checkout
FR_1499,dairy,dairy
FR_1499,dairy,dairy


In [23]:
df_mar.to_csv('df_mar.csv')

In [24]:
P = pd.crosstab(df_mar['before'], df_mar['after'], normalize=0)
P

after,checkout,dairy,drinks,fruit,spices
before,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.103466,0.736750,0.058569,0.049848,0.051367
drinks,0.215710,0.010899,0.598499,0.087909,0.086983
fruit,0.201605,0.095924,0.054847,0.596947,0.050677
spices,0.150685,0.193214,0.163109,0.090953,0.402039


In [25]:
states = ['checkout','dairy','drinks',"fruit",'spices']

In [26]:
random.choice(states)


'fruit'

In [27]:
# convery the transition matrix to a dictionary that maps state to a list of probabilities
# order such that the first value is transition probability to cold and the second value
# is transition probability to hot

probs = P.to_dict(orient='index')
#probs

In [28]:
# convert internal values, which is a dict to a list
for key in probs.keys():
    probs[key] = list(probs[key].values())



In [29]:
probs

{'checkout': [1.0, 0.0, 0.0, 0.0, 0.0],
 'dairy': [0.10346573646899966,
  0.7367503094407561,
  0.05856869584786767,
  0.04984809271970294,
  0.05136716552267357],
 'drinks': [0.2157104667900473,
  0.01089862224964014,
  0.5984988690108987,
  0.0879086983343615,
  0.08698334361505243],
 'fruit': [0.20160528800755428,
  0.09592382751022978,
  0.05484734025810513,
  0.5969468051621026,
  0.05067673906200818],
 'spices': [0.1506849315068493,
  0.19321439949028352,
  0.163109270468302,
  0.09095253265371138,
  0.4020388658808538]}

In [ ]:
# class Customer1:
#     """
#     a single customer that moves through the supermarket
#     in a MCMC simulation
#     """

#     def __init__(self, id, state, transition_probs, budget=100):
#         self.id = id
#         self.state = state
#         self.transition_probs = transition_probs
#         self.budget = budget

#     def __repr__(self):
#         return f'<Customer_no {self.id} in {self.state}>'

#     def next_state(self):
#         '''
#         Propagates the customer to the next state.
#         Returns nothing.
#         '''
#         #self.state = random.choice(['spices', 'drinks', 'fruit'])
#         self.state = random.choices(states, weights=self.transition_probs[self.state])[0]
#         return self.state

#     def is_active(self):
#         """Returns True if the customer has not reached the checkout yet."""
#         return self.state != 'checkout'



In [ ]:
# bagigio = Customer1('bagigio','dairy',probs)

In [ ]:
# lst_customers = [bagigio]

In [ ]:
# for i in lst_customers:
#     i = 0   
#     min_time = random_time(text=True, pattern='%H:%M:%S')
#     # create datetime object from timestamp string
#     given_time = datetime.strptime(min_time, "%H:%M:%S")
#     print(type(given_time))
#     print(f"{given_time.time()} -- {bagigio.id} is in {bagigio.state} section")
#     print(type(given_time))
#     while bagigio.is_active():    
#         i+=1
#         print(f"{(given_time + timedelta(minutes = i)).time()} -- {bagigio.id} is in {bagigio.next_state()} section")


In [30]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """

    def __init__(self, name, state, budget=100):
        self.name = name
        self.state = state
        self.budget = budget

    def __repr__(self):
        return f'<Customer_no {self.id} in {self.state}>'

    def is_active(self):
        """Returns True if the customer has not reached the checkout yet."""
        return self.state != 'checkout'
    
    def is_not_active(self):
        """Returns True if the customer has not reached the checkout yet."""
        return self.state == 'checkout'

    


In [ ]:
"""
Start with this to implement the supermarket simulator.
"""


class Supermarket:
    """manages multiple Customer instances that are currently in the market.
    """

    def __init__(self, opening, closing):
        ### ADD OPENING AND CLOSING HOURS
        # a list of Customer objects
        self.dict_csv = {'timestamp':[],'customer_no':[],'location':[]}
        self.customers = []
        self.customers_timers = []
        self.time = (datetime.strptime(opening, "%H:%M"))
        self.end_time = (datetime.strptime(closing, "%H:%M"))

    def __repr__(self):
        return 'something about the supermarket'

    def get_time(self):
        """current time in HH:MM format,
        """
        return f"It's {self.time.time()}"
        # ??? just get back the current time

    def print_customers(self):
        """print all customers with the current time and id in CSV format.
        """
        df = pd.DataFrame(self.dict_csv)
        df.to_csv("supermarket.csv")
        return df


        # return self.d_customers_no, self.d_timestamp, self.d_location

    def next_minute(self):
        """propagates all customers to the next state.
        """
        minute = 0
        #if str(self.time.time()) == str((self.end_time - timedelta(minutes=5)).time()):


        while str(self.time.time()) != str(self.end_time.time()):
            self.time = (self.time + timedelta(minutes=minute))
            #print(str(self.time.time())[:-3])
            
            print()
            for i in range(self.number_clients):
                if self.customers_timers[i] == str(self.time.time())[:-3]:
                    self.dict_csv['timestamp'].append(self.customers_timers[i])
                    self.dict_csv['customer_no'].append(self.customers[i].id)
                    self.dict_csv['location'].append(self.customers[i].state)
                    internal_min = 0
                    while self.customers[i].is_active():
                        print(internal_min)
                        self.customers[i].state = random.choices(
                            states, weights=probs[self.customers[i].state])[0]
                        internal_min += 1
                        self.dict_csv['timestamp'].append(str((self.time + timedelta(minutes=internal_min)).time())[:-3])
                        self.dict_csv['customer_no'].append(self.customers[i].id)
                        self.dict_csv['location'].append(self.customers[i].state)

                        
                else:
                    continue
                    

            
            minute = 1


    def add_new_customers(self):
        """randomly creates new customers.
        """
        self.number_clients = random.randint(100, 200)
        for i in range((self.number_clients)):
            self.customers.append(Customer(i, random.choice(states[1:])))
            self.customers_timers.append(random_time(start=self.time.time(), end=self.end_time.time(), text=True, pattern="%H:%M"))

        
        



In [39]:
"""
Start with this to implement the supermarket simulator.
"""

class Supermarket_01:
    """manages multiple Customer instances that are currently in the market.
    """

    def __init__(self, opening, closing):
        self.dict_csv = {'timestamp':[],'customer_name':[],'location':[]}
        self.customers = []
        self.time = (datetime.strptime(opening, "%H:%M"))
        self.end_time = (datetime.strptime(closing, "%H:%M"))

    def __repr__(self):
        return 'something about the supermarket'

    def get_time(self):
        """current time in HH:MM format,
        """
        return f"It's {self.time.time()}"

    def print_customers(self):
        """print all customers with the current time and id in CSV format.
        """
        df = pd.DataFrame(self.dict_csv)
        df.to_csv("supermarket_01.csv")
        return df, self.dict_csv

    def next_minute(self):
        """propagates all customers to the next state.
        """
        minute = 0
        
        while str(self.time.time()) != str(self.end_time.time()):
            self.time = (self.time + timedelta(minutes=minute))
            
            r_range = int(np.sqrt(random.randint(0,len(self.customers))))

            for r in range(r_range):
                if self.customers[r].is_active():
                    self.write_on_dict(r)
                    self.customers[r].state = random.choices(
                        states, weights=probs[self.customers[r].state])[0]

                elif self.customers[r].is_not_active():
                    self.write_on_dict(r)
                    self.customers.pop(r)
                
                    if len(self.customers) == 0:
                        break
                    else:
                        continue 

            minute = 1
                
    def write_on_dict(self, r):
        self.dict_csv['timestamp'].append(str(self.time.time()))
        self.dict_csv['customer_name'].append(self.customers[r].name)
        self.dict_csv['location'].append(self.customers[r].state)
        print(
            f'{str(self.time.time())} {self.customers[r].name} in {self.customers[r].state}')

    def add_new_customers(self):
        """randomly creates new customers.
        """
        self.number_clients = random.randint(3, 3)
        #print(self.number_clients)
        for i in range((self.number_clients)):
            self.customers.append(
                Customer(names.get_full_name(), random.choice(states[1:])))
        
        print(len(self.customers))

    



In [40]:
s = Supermarket_01('07:00','22:00')

In [41]:
s.add_new_customers()

3


In [42]:
s.next_minute()

07:00:00 Velma Gadson in dairy
07:01:00 Velma Gadson in dairy
07:03:00 Velma Gadson in dairy
07:04:00 Velma Gadson in dairy
07:05:00 Velma Gadson in dairy
07:06:00 Velma Gadson in dairy
07:07:00 Velma Gadson in dairy
07:08:00 Velma Gadson in dairy
07:10:00 Velma Gadson in dairy
07:12:00 Velma Gadson in spices
07:13:00 Velma Gadson in dairy
07:14:00 Velma Gadson in checkout
07:15:00 Erika Thomas in dairy
07:16:00 Erika Thomas in dairy
07:17:00 Erika Thomas in dairy
07:18:00 Erika Thomas in checkout
07:19:00 Anna Bartamian in spices
07:21:00 Anna Bartamian in spices
07:22:00 Anna Bartamian in checkout


In [43]:
df, dict = s.print_customers()

In [44]:
df.head(40)

,timestamp,customer_name,location
0,07:00:00,Velma Gadson,dairy
1,07:01:00,Velma Gadson,dairy
2,07:03:00,Velma Gadson,dairy
3,07:04:00,Velma Gadson,dairy
4,07:05:00,Velma Gadson,dairy
5,07:06:00,Velma Gadson,dairy
6,07:07:00,Velma Gadson,dairy
7,07:08:00,Velma Gadson,dairy
8,07:10:00,Velma Gadson,dairy
9,07:12:00,Velma Gadson,spices


In [45]:
# dict

In [51]:
import cv2
import time
import random
import numpy as np
import pandas as pd


TILE_SIZE = 32

MARKET = """
##WWWWWWWWWWWWWWW##
#tttttttttttttttttG
rRtDeeDtSssStFbaFt#
dRtDmmDtSppStFbaFt#
rRtDeeDtSssStFbaFt#
dRtDmmDtSppStFbaFt#
rRtDeeDtSssStFbaFt#
rRtttttttttttttttt#
CCccCCccCCccCCtttt#
##tttttttttttttttt#
##tttttttttttttttt#
##ttEEttEEttEEttE##
""".strip()


class SupermarketMap:
    """Visualizes the supermarket background"""

    def __init__(self, layout, tiles):
        """
        layout : a string with each character representing a tile
        tiles   : a numpy array containing all the tile images
        """
        self.tiles = tiles
        # split the layout string into a two dimensional matrix [['#','#',..],['#','#',.]]
        self.contents = [list(row) for row in layout.split("\n")]
        self.ncols = len(self.contents[0])
        self.nrows = len(self.contents)
        # create a 3d np.zeros image with shape (384, 576, 3)
        self.image = np.zeros(
            (self.nrows*TILE_SIZE, self.ncols*TILE_SIZE, 3), dtype=np.uint8
        )
        self.prepare_map()

    def extract_tile(self, row, col):
        """extract a tile array from the tiles image"""
        # we get tile array inserting the row and col in the tiles.png where our tile is
        y = row*TILE_SIZE
        x = col*TILE_SIZE
        return self.tiles[y:y+TILE_SIZE, x:x+TILE_SIZE]

    def get_tile(self, char):
        """returns the array for a given tile character"""
        if char == "#":
            return self.extract_tile(0, 0)
        elif char == "G":
            return self.extract_tile(7, 3)
        elif char == "C":
            return self.extract_tile(2, 8)
        elif char == "b":
            return self.extract_tile(0, 4)
        elif char == "a":
            return self.extract_tile(5, 4)
        elif char == "s":
            return self.extract_tile(5, 9)
        elif char == "p":
            return self.extract_tile(6, 9)
        elif char == "e":
            return self.extract_tile(-2, -5)
        elif char == "m":
            return self.extract_tile(-4, 6)
        elif char == "d":
            return self.extract_tile(3, -3)
        elif char == "r":
            return self.extract_tile(-3, -3)
        elif char == "W":
            return self.extract_tile(0, 7)
        elif char == "E":
            return self.extract_tile(-3, -6)
        elif char == "p":
            return self.extract_tile(3, 2)
        else:
            return self.extract_tile(1, 2)

    def prepare_map(self):
        """prepares the entire image as a big numpy array"""
        # this for loops goes through the MARKET char's layout, and it creates the image
        for row, line in enumerate(self.contents):
            for col, char in enumerate(line):
                # bm give back a real tile from tiles.png
                bm = self.get_tile(char)
                y = row*TILE_SIZE
                x = col*TILE_SIZE
                self.image[y:y+TILE_SIZE, x:x+TILE_SIZE] = bm

    def draw(self, frame):
        """
        draws the image into a frame
        """
        ###
        frame[0:self.image.shape[0], 0:self.image.shape[1]] = self.image

    def write_image(self, filename):
        """writes the image into a file"""
        cv2.imwrite(filename, self.image)


class Customer_map:

    def __init__(self, name, supermarket, avatar, row, col, state=None):
        """
        supermarket: A SuperMarketMap object
        avatar : a numpy array containing a 32x32 tile image
        row: the starting row
        col: the starting column
        """

        self.name = name
        self.supermarket = supermarket
        self.state = state
        self.avatar = avatar
        self.row = row
        self.col = col

    def draw(self, frame):
        x = self.col * TILE_SIZE
        y = self.row * TILE_SIZE
        frame[y:y+TILE_SIZE, x:x+TILE_SIZE] = self.avatar

    def move(self):
        #states = ['checkout', 'dairy', 'drinks', "fruit", 'spices']
        dairy = [(2, 3), (2, 6), (3, 3), (3, 6), (4, 3),
                 (4, 6), (5, 3), (5, 6), (6, 3), (6, 6)]
        fruits = [(2, 13), (2, 16), (3, 13), (3, 16), (4, 13),
                  (4, 16), (5, 13), (5, 16), (6, 13), (6, 16)]
        drinks = [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
        spices = [(2, 8), (2, 11), (3, 8), (3, 11), (4, 8),
                  (4, 11), (5, 8), (5, 11), (6, 8), (6, 11)]
        checkout = [(8, 2), (8, 3), (8, 6), (8, 7), (8, 10), (8, 11)]

        
        ### Calculate the difference between now and the random choice
        ### moving in that direction with for loop in order to reach that column and row

        if self.state == 'fruit':
            self.row, self.col = random.choice(fruits)

        if self.state == 'checkout':
            self.row, self.col = random.choice(checkout)

        if self.state == 'spices':
            self.row, self.col = random.choice(spices)

        if self.state == 'dairy':
            self.row, self.col = random.choice(dairy)

        if self.state == 'drinks':
            self.row, self.col = random.choice(drinks)

    # def move_checkout(self):
    #     new_row = self.row
    #     new_col = self.col
    #     new_row += 3
    #     time.sleep(0.75)
            
    #     if self.supermarket.contents[new_row][new_col] == 't':
    #         self.col = new_col
    #         self.row = new_row


if __name__ == "__main__":

    # we are creating a bigger zeros array as background for our image
    background = np.zeros((500, 700, 3), np.uint8)
    tiles = cv2.imread("tiles.png")
    
    market = SupermarketMap(MARKET, tiles)
    
    df = pd.read_csv("Supermarket_01.csv", index_col=1, parse_dates=True)
    list_of_avatars = {}
    possible_avatar_pos = [(7, 0), (7, 15), (8, 15), (6, 15), (5, 14)]
    for n in range(len(list(df.customer_name.unique()))):
        r = random.choice(possible_avatar_pos) 
        no = list(df.customer_name.unique())[n]
        customer = Customer_map(name=no, supermarket=market, avatar=market.extract_tile(r[0],r[1]), row=1, col=18)
        list_of_avatars[no] = customer

    # while True:
    lst_timestamp = dict['timestamp']
    print(lst_timestamp)
    lst_customer_name = dict['customer_name']
    print(lst_customer_name)
    lst_location = dict['location']  
    print(lst_location)
    
    for l in list_of_avatars:
        ind = lst_customer_name.index(l)
        list_of_avatars[l].state = lst_location[ind]

    
    for i in range(len(lst_customer_name)):
        nome = lst_customer_name[i]
        frame = background.copy()
        market.draw(frame)        

        for l in list_of_avatars:
            list_of_avatars[l].draw(frame)
                
            if l==nome:
                location = lst_location[i]
                list_of_avatars[lst_customer_name[i]].state = location
                list_of_avatars[lst_customer_name[i]].draw(frame)
                list_of_avatars[lst_customer_name[i]].move()
            
            

            if list_of_avatars[l].state == 'checkout':

                list_of_avatars[l].row += 3 
                list_of_avatars[l].row = 11
            
            # https://www.ascii-code.com/
            time.sleep(1)

            cv2.imshow("frame", frame)
        print(f'{lst_timestamp[i]} -- {lst_customer_name[i]} in {lst_location[i]}')

        key = cv2.waitKey(1)

        if key == 113:  # 'q' key
            break

        cv2.imshow("frame", frame)

    cv2.destroyAllWindows()

    market.write_image("supermarket.png")


['07:00:00', '07:01:00', '07:03:00', '07:04:00', '07:05:00', '07:06:00', '07:07:00', '07:08:00', '07:10:00', '07:12:00', '07:13:00', '07:14:00', '07:15:00', '07:16:00', '07:17:00', '07:18:00', '07:19:00', '07:21:00', '07:22:00']
['Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Velma Gadson', 'Erika Thomas', 'Erika Thomas', 'Erika Thomas', 'Erika Thomas', 'Anna Bartamian', 'Anna Bartamian', 'Anna Bartamian']
['dairy', 'dairy', 'dairy', 'dairy', 'dairy', 'dairy', 'dairy', 'dairy', 'dairy', 'spices', 'dairy', 'checkout', 'dairy', 'dairy', 'dairy', 'checkout', 'spices', 'spices', 'checkout']
07:00:00 -- Velma Gadson in dairy
07:01:00 -- Velma Gadson in dairy
07:03:00 -- Velma Gadson in dairy
07:04:00 -- Velma Gadson in dairy
07:05:00 -- Velma Gadson in dairy
07:06:00 -- Velma Gadson in dairy
07:07:00 -- Velma Gadson in dairy
07:08:00 -- Velma Gadson in dairy
07:1